In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items)) 
        
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 리스트를 생성합니다. 값은 콤마로만 구분해주세요."),
    ("system", "리스트 크기는 {max_length}를 넘을 수 없고, 값은 중복될 수 없습니다."),
    ("human", "{question}")
])

# prompt = template.format_messages(max_length=10, question="국가들을 알려주세요." )
# result = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(result.content)

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_length":5,
    "question":"태국의 도시들을 알려주세요."
})

In [8]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 월드클래스 요리사입니다. 당신은 구하기 쉬운 재료로 쉽게 따라할 수 있는 요리 레시피를 만들 수 있습니다."),
    ("human", "저는 {country} 요리를 만들고 싶습니다.")
])
chef_chain = chef_prompt | chat


veg_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 월드클래스 채식 요리사입니다. 당신은 전달 받은 레시피를 통해 채식 요리 레시피로 변경할 수 있습니다."),
    ("system", "대체 재료를 명확히 찾을 수 없거나, 레시피를 너무 많이 바꿔야 한다면 할 수 없다고 말해야 합니다."),
    ("human", "{recipe}")
])

veg_chain = veg_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain
result = final_chain.invoke({"country":"한국"}).content

print(result)


좋아요! 한국 요리 중에서 가장 유명한 요리 중 하나는 불고기입니다. 불고기는 소고기를 달달한 소스에 재워 구워낸 요리로 매우 맛있고 간단하게 만들 수 있습니다. 이 레시피를 따라해보세요!

**불고기 레시피**

**재료:**
- 소고기 (양념용 불고기용 갈은 소고기) 500g
- 양파 1개
- 당근 1/2개
- 대파 1대
- 마늘 4쪽
- 간장 4큰술
- 설탕 2큰술
- 참기름 1큰술
- 깨소금 약간
- 후추 약간
- 식용유

**만드는 법:**
1. 양파, 당근, 대파를 얇게 채 썰어 준비합니다.
2. 마늘은 다지거나 간 마늘을 사용합니다.
3. 볼에 간장, 설탕, 참기름, 깨소금, 후추, 다진 마늘을 넣고 잘 섞어 소스를 만듭니다.
4. 소고기를 소스에 넣고 버무려 10분 정도 재웁니다.
5. 팬에 식용유를 두르고 소고기와 채소를 넣어 볶아줍니다.
6. 고기가 익으면 대파를 넣고 한 번 더 볶아줍니다.
7. 불고기를 그릇에盛り付け하여 마무리합니다.

이렇게 간단하게 만들 수 있는 불고기 요리를 즐기세요! 맛있게 드십시오.좋습니다! 불고기 레시피를 채식 요리 레시피로 변경해보겠습니다.

**채식 불고기 레시피**

**재료:**
- 두부 500g
- 양파 1개
- 당근 1/2개
- 대파 1대
- 마늘 4쪽
- 간장 4큰술
- 설탕 2큰술
- 참기름 1큰술
- 깨소금 약간
- 후추 약간
- 식용유

**만드는 법:**
1. 양파, 당근, 대파를 얇게 채 썰어 준비합니다.
2. 마늘은 다지거나 간 마늘을 사용합니다.
3. 볼에 간장, 설탕, 참기름, 깨소금, 후추, 다진 마늘을 넣고 잘 섞어 소스를 만듭니다.
4. 두부를 소스에 넣고 버무려 10분 정도 재웁니다.
5. 팬에 식용유를 두르고 두부와 채소를 넣어 볶아줍니다.
6. 두부가 익으면 대파를 넣고 한 번 더 볶아줍니다.
7. 채식 불고기를 그릇에盛り付け하여 마무리합니다.

소고기 대신 두부를 사용하여 채식 불고기를 만들었습니다. 소스와 채소는 원본 레시피와 동일하게 사용하였습니다. 맛있게 즐기세요!좋습